In [1]:
import numpy as np
import dask
import fastprogress
from dask.distributed import Client

In [2]:
client = Client(threads_per_worker=1, n_workers=64)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 64
Total threads: 64,Total memory: 251.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45819,Workers: 64
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 251.57 GiB
Comm: tcp://127.0.0.1:34929,Total threads: 1
Dashboard: http://127.0.0.1:40797/status,Memory: 3.93 GiB
Nanny: tcp://127.0.0.1:42391,


In [3]:
np.random.seed = 42

In [4]:
n = 1260
num_tribes = 5
clan_size = 7
tribe_size = n // num_tribes
num_clans_in_tribe = tribe_size // clan_size
num_clans_in_tribe, num_clans_in_tribe / 4

(36, 9.0)

In [5]:
def exp(num_runs,
        n = 1260,
        num_tribes = 5,
        f = 0.333333, # global Byzantine node ratio
        clan_size = 7, # number of nodes in a clan
        tribe_threshold = None,
        clan_threshold = None,
       ):

    total_failures = 0

    tribe_size = n // num_tribes # number of nodes in a tribe
    num_clans = tribe_size // clan_size # number of clans in a tribe
    assert n == clan_size * num_clans * num_tribes
    
    num_byzantine_nodes = int(n * f) # total number of Byzantine nodes in the entire Network, including all tribes

    if tribe_threshold is None:
        tribe_f = num_clans // 3
        if num_clans == tribe_f * 3:
            tribe_f = tribe_f - 1
        tribe_threshold = tribe_f * 2 + 1 # Assumption: when over 2/3 clans are Byzantine, then the tribe fails

    if clan_threshold is None:
        clan_f = clan_size // 3
        if clan_f == clan_f * 3:
            clan_f = clan_f - 1
        clan_threshold = clan_f * 2 + 1 # Assumption: when over 2/3 nodes in a clan are Byzantine, then the clan becomes Byznantine
        
    for i in range(num_runs):        
        node_is_byzantine = np.concatenate((np.ones(num_byzantine_nodes, dtype=np.bool), # 1 means this node is Byzantine
                                    np.zeros(n - num_byzantine_nodes, dtype=np.bool))) # 0 means this node is honest
        np.random.shuffle(node_is_byzantine) # Byzantine nodes are randomly scattered
        structured_node_is_byzantine = node_is_byzantine.reshape([-1, num_clans, clan_size]) # (tribe, clan, node) structure
        clan_num_byzantine_nodes = structured_node_is_byzantine.sum(axis=2) # number of Byzantine nodes in each clan

        clan_is_byzantine = clan_num_byzantine_nodes >= clan_threshold
        num_byzantine_clans = clan_is_byzantine.sum(axis=1) # number of Byzantine clans in each tribe
        
        num_byzantine_tribes = num_byzantine_clans >= tribe_threshold # whether a tribe is Byzantine
        
        total_failures += (num_byzantine_tribes.sum() == num_tribes) # a simulation fails if all tribe fail
    return total_failures

In [6]:
num_simulations = 10000000
num_tasks = 1000
num_runs = num_simulations // num_tasks

lazy_result_total = 0
for i in fastprogress.progress_bar(range(num_tasks)):
    lazy_result = dask.delayed(exp)(num_runs=num_runs, f=0.33, clan_threshold=3, tribe_threshold=10)
    lazy_result_total += lazy_result

In [7]:
result = dask.compute(lazy_result_total) # http://server_ip:8787
result[0] / num_simulations

0.9078683

In [8]:
num_simulations = 10000000
num_tasks = 1000
num_runs = num_simulations // num_tasks

lazy_result_total = 0
for i in fastprogress.progress_bar(range(num_tasks)):
    lazy_result = dask.delayed(exp)(num_runs=num_runs, f=0.25, clan_threshold=3, tribe_threshold=10)
    lazy_result_total += lazy_result
    
result = dask.compute(lazy_result_total) # http://server_ip:8787
result[0] / num_simulations

0.000493

In [9]:
num_simulations = 10000000
num_tasks = 1000
num_runs = num_simulations // num_tasks

lazy_result_total = 0
for i in fastprogress.progress_bar(range(num_tasks)):
    lazy_result = dask.delayed(exp)(num_runs=num_runs, f=0.2, clan_threshold=3, tribe_threshold=10)
    lazy_result_total += lazy_result
    
result = dask.compute(lazy_result_total) # http://server_ip:8787
result[0] / num_simulations

0.0